## Scaping Psychology Today for Mental Health Professionals in NYC

### Load libraries

In [1]:
print "importing libraries..."
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import lxml
import pandas as pd
from bs4 import BeautifulSoup

## Lets get scraping!

In [6]:
# define target url
url = 'https://therapists.psychologytoday.com/rms/county/NY/New+York.html'
driver = webdriver.Chrome()  #select selenium web driver
docs = []                   #generating empty list
while True:
    time.sleep(1)           #added a 1 second sleep to limit bot detection
    driver.get(url)         #open the url in selenium

    soup = BeautifulSoup(driver.page_source,'lxml') #grab the content with beautifulsoup for parsing
    main_table = soup.findAll('div',{'class':'no-min-height'})[0]  # select the desired html node
    docs_per_page = len(main_table.contents[2].contents[5].contents[1].findAll('div',{'class':'row'},recursive=False)) # outputs number of html node chldren - aka doctors per web page
    docs += [main_table.contents[2].contents[5].contents[1].contents[i].contents[1].contents[1].attrs for i in range(1,docs_per_page*2,2)] #building a list of dictionaries - filled with doctor info
    next_page_txt = main_table.contents[2].contents[7].contents[1].contents[3].contents[1].find_all('a',href=True)[~0].text #next page button should have text == 'Next' except for final page
    next_page_url = main_table.contents[2].contents[7].contents[1].contents[3].contents[1].find_all('a',href=True)[~0]['href'] #outputs url for next page of content
    if next_page_txt == 'Next':  #if a 'next page' exists...
        url = next_page_url       # set url = next page url to prepare for next page of scraping
    else:
        break                    #scraping complete!
        print "Scraping Complete!"

importing libraries...


### clean up scrapped data and save it to csv

In [25]:
doc_df = pd.DataFrame(docs).drop(['class','data-new-clients','style'],axis=1).dropna(axis=0) # turn list of dicts to data frame, drop unimportant columns, remove na values
docs_1 = doc_df[['data-phone','data-prof-name','data-profid']] #select columns phone #, name, id # 
docs_1.columns = ['Phone Number', 'Physician Name', 'Psychology Today id'] #give columns more appropriate names
docs_l = docs_1.sort_values(by='Physician Name', ascending=True).reset_index().drop(['index'],axis=1) # sorts data by Name, resets out of order index
docs_l.to_csv('NYC-mental-health-professionals-2017.csv',encoding='utf8')  #save output to .csv file

In [26]:
docs_1

,Phone Number,Physician Name,Psychology Today id
0,(917) 746-8647,Deborah V Kohloss,116139
1,(646) 386-2128,Darcey J Ortolf,51317
2,(917) 300-1896,Ariela Vasserman,113648
3,(347) 354-2169,Tiffany C Miller,81997
4,(646) 461-4924,Erica Komisar,103566
5,(646) 398-2962,Jessica Hirsh Weiss,43208
6,(917) 746-1534,Renee Obstfeld,43318
7,(773) 377-8559,Katherine Jenness,210331
8,(347) 744-9481,Matthew James Tirelli,207885
10,(646) 663-4304,Lucy Bradley Smith,174295
